# FAERS graph

## Data quality analysis

### 1. Introduction

This notebook checks the data contents and quality of the quarterly FAERS data files, available for download [here](https://fis.fda.gov/extensions/FPD-QDE-FAERS/FPD-QDE-FAERS.html).  

The data can be downloaded in two formats: XML and ASCII. Each of these data downloads contains identical README and FAQ documentation pdfs, along with the data files and documentation pdfs specific to the two data formats. The latter contain total record counts, missing value counts for selected fields, and frequency counts for categorical values.  

According to the README.pdf doc that comes with the data downloads, the two data formats mostly contain the same data, but each has some column that the other doesn't. 

In this notebook we'll take a look at what the available data formats look like. We can start with processing whichever data format is easier to work with, and add any extra fields from the other format if needed later.  
We'll check the data for consistency with the counts, missing value numbers and frequencies reported by FDA.  
We'll also check the data for any general anomalies, and not what data cleaning will need to be done.

#### Notebook contents:
1. [Introduction](#1.-Introduction)
2. [Notebook setup](#2.-Notebook-setup)
3. [Data sources](#3.-Data-sources)  
4. [Sample raw data files](#4.-Sample-raw-data-files)  
    4.1 [XML data files](#4.1-XML-data-files)  
    4.2 [ASCII data files](#4.2-ASCII-data-files)  
    4.2.1 [DEMO file](#4.2.1-DEMO-file)  
    = [Summary for DEMO ASCII file](#Summary-for-DEMO-ASCII-file)
5. [DQA summary](#5.-DQA-summary)
6. [Next steps](#6.-Next-steps)

### 2. Notebook setup  
#### Imports

In [1]:
import pandas as pd
import numpy as np

import re
import xml.etree.ElementTree as ET

from timeit import default_timer as timer

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

#### Settings

In [2]:
# Customize matplotlib default settings
matplotlib.rcParams.update({'font.size': 16})
plt.rcParams["figure.figsize"] = (20,10)

In [3]:
# set up Pandas options
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)
pd.options.display.float_format = '{:.2f}'.format

#### Helper functions

In [338]:
def show_value_counts(ser, topn="none", sort_on="freq", sort_ascending=False):
    """Example usage: show_value_counts(demo.caseversion, 5, sort_on="labels", sort_ascending=True)"""
    counts = ser.value_counts(dropna=False)
    normalized = ser.value_counts(normalize=True, dropna=False)
    
    if sort_on == "freq" and sort_ascending == True:
        counts.sort_values(ascending=sort_ascending, inplace=True, na_position='first')
        normalized.sort_values(ascending=sort_ascending, inplace=True, na_position='first')
    elif sort_on == "labels":
        counts.sort_index(ascending=sort_ascending, inplace=True, na_position='first')
        normalized.sort_index(ascending=sort_ascending, inplace=True, na_position='first')
        
    if topn != "none":
        counts = counts.head(topn)
        normalized = normalized.head(topn)
        
    df = pd.concat([counts, normalized], axis=1).reset_index()
    
    df.columns = [ser.name, "count", "proportion"]
    
    return df
        

In [349]:
def show_na(ser):
    # total records
    l = len(ser)
    # missing values count
    m = ser.isna().sum()
    
    return pd.DataFrame([{'na_count': m, 'na_proportion': m/l}])

### 3. Data sources

FAERS stands for FDA Adverse Event Reporting System. It is a database that contains adverse event reports, medication error reports and product quality complaints resulting in adverse events that were submitted to FDA. The database is designed to support the FDA's post-marketing safety surveillance program for drug and therapeutic biologic products. ([Source](https://www.fda.gov/Drugs/GuidanceComplianceRegulatoryInformation/Surveillance/AdverseDrugEffects/default.htm)) 


https://www.fda.gov/Drugs/GuidanceComplianceRegulatoryInformation/Surveillance/AdverseDrugEffects/ucm082193.htm 



### 4. Sample raw data files  


Datafile download for 2018Q4.

In [3]:
raw_data_path = "data/raw/"

In [4]:
!ls data/raw/

ascii_2018q4 xml_2018q4


#### 4.1 XML data files 
Let's look at the XML data format. We can parse this file format with the xml ElementTree library.  
I've also tried using BeautifulSoup for parsing these files, but with xml parser it ran extremely slow.

In [5]:
!ls data/raw/xml_2018q4/xml

1_ADR18Q4.xml 2_ADR18Q4.xml 3_ADR18Q4.xml XML_NTS.pdf   xml18q4.pdf


In [5]:
! head -50 data/raw/xml_2018q4/xml/1_ADR18Q4.xml

<?xml version="1.0"?>
<ichicsr lang="en">
  <ichicsrmessageheader>
    <messagetype>ICSR</messagetype>
    <messageformatversion>2.1</messageformatversion>
    <messageformatrelease>1.0</messageformatrelease>
    <messagenumb>2019-02</messagenumb>
    <messagesenderidentifier>FDA CDER</messagesenderidentifier>
    <messagereceiveridentifier>Public Use</messagereceiveridentifier>
    <messagedateformat>204</messagedateformat>
    <messagedate>20190207040220</messagedate>
  </ichicsrmessageheader>
  <safetyreport>
    <safetyreportversion>1</safetyreportversion>
    <safetyreportid>15529521</safetyreportid>
    <primarysourcecountry>US</primarysourcecountry>
    <occurcountry>US</occurcountry>
    <transmissiondateformat>102</transmissiondateformat>
    <transmissiondate>20190205</transmissiondate>
    <reporttype>1</reporttype>
    <serious>2</serious>
    <receivedateformat>102</receivedateformat>
    <receivedate>20181018</receivedate>
    <receiptdateformat>102</receiptdateformat>
  

In [6]:
raw_xml_path = "data/raw/xml_2018q4/xml/"

In [7]:
xml_file_1 = raw_xml_path + "1_ADR18Q4.xml"

In [8]:
tree = ET.parse(xml_file_1)
root = tree.getroot()

In [9]:
root.tag

'ichicsr'

In [10]:
root.attrib

{'lang': 'en'}

In [11]:
i=0
for child in root:
    print(child.tag, child.attrib)
    i+=1
    if i>10:
        break

ichicsrmessageheader {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}
safetyreport {}


In [12]:
root[0][1].text

'2.1'

In [13]:
i=0
        
for report_id in root.iter('safetyreportid'):
    print(report_id.text)
    i+=1
    if i>10:
        break

15529521
15529522
15529524
15529856
15529858
15529861
15530134
15529556
15529558
15529559
15529564


In [14]:
all_tags = list(set([elem.tag for elem in root.iter()]))

In [15]:
len(all_tags)

87

In [16]:
all_tags.sort()

Print out all data fields in this XML file

In [17]:
all_tags

['actiondrug',
 'activesubstance',
 'activesubstancename',
 'authoritynumb',
 'companynumb',
 'drug',
 'drugadditional',
 'drugadministrationroute',
 'drugauthorizationnumb',
 'drugbatchnumb',
 'drugcharacterization',
 'drugcumulativedosagenumb',
 'drugcumulativedosageunit',
 'drugdosageform',
 'drugdosagetext',
 'drugenddate',
 'drugenddateformat',
 'drugindication',
 'drugintervaldosagedefinition',
 'drugintervaldosageunitnumb',
 'drugrecuraction',
 'drugrecurreadministration',
 'drugrecurrence',
 'drugseparatedosagenumb',
 'drugstartdate',
 'drugstartdateformat',
 'drugstructuredosagenumb',
 'drugstructuredosageunit',
 'drugtreatmentduration',
 'drugtreatmentdurationunit',
 'duplicate',
 'duplicatenumb',
 'duplicatesource',
 'fulfillexpeditecriteria',
 'ichicsr',
 'ichicsrmessageheader',
 'literaturereference',
 'medicinalproduct',
 'messagedate',
 'messagedateformat',
 'messageformatrelease',
 'messageformatversion',
 'messagenumb',
 'messagereceiveridentifier',
 'messagesenderiden

#### 4.2 ASCII data files

Now let's look at the ASCII datafiles format. According to the docs, these files are delimiter-separated text files, with the delimiter being `$`. The data is split up into separate files that correspond to database tables and are organized around their respective primary keys.

In [18]:
!ls data/raw/ascii_2018q4/ascii

ASC_NTS.pdf  INDI18Q4.txt RPSR18Q4.txt drug18q4.pdf reac18q4.pdf
DEMO18Q4.txt OUTC18Q4.txt THER18Q4.txt indi18q4.pdf rpsr18q4.pdf
DRUG18Q4.txt REAC18Q4.txt demo18q4.pdf outc18q4.pdf ther18q4.pdf


#### 4.2.1 DEMO file

In [19]:
!head data/raw/ascii_2018q4/ascii/DEMO18Q4.txt

primaryid$caseid$caseversion$i_f_code$event_dt$mfr_dt$init_fda_dt$fda_dt$rept_cod$auth_num$mfr_num$mfr_sndr$lit_ref$age$age_cod$age_grp$sex$e_sub$wt$wt_cod$rept_dt$to_mfr$occp_cod$reporter_country$occr_country
100035916$10003591$6$F$20130718$20181203$20140312$20181211$EXP$$PHHY2013GB101660$NOVARTIS$$47$YR$$F$Y$$$20181211$$OT$GB$GB
100050413$10005041$3$F$20140306$20141118$20140312$20181213$EXP$$US-TEVA-468475USA$TEVA$$25$YR$$F$Y$68.1$KG$20181213$$CN$US$US
1000551312$10005513$12$F$20120209$20181107$20140313$20181115$EXP$$BR-AMGEN-BRASP2012013548$AMGEN$$55$YR$A$F$Y$67$KG$20181115$$CN$BR$BR
100058832$10005883$2$F$$20180928$20140313$20181012$EXP$$FR-RANBAXY-2014RR-78735$RANBAXY$$31$YR$$F$Y$$$20181012$$OT$GB$FR
100065479$10006547$9$F$201203$20181211$20140313$20181228$EXP$$US-BAYER-2014-035909$BAYER$$36$YR$A$F$Y$90.7$KG$20181228$$CN$US$US
100066188$10006618$8$F$$20181004$20140313$20181017$PER$$US-PFIZER INC-2014069077$PFIZER$$58$YR$$F$Y$$$20181017$$CN$US$US
1000808588$10008085$88$F$20131213$2

In [20]:
raw_ascii_path = "data/raw/ascii_2018q4/ascii/"

In [21]:
ascii_file_demo = raw_ascii_path + "DEMO18Q4.txt"

In [22]:
datatypes = {
    'primaryid': 'object', 
    'caseid': 'object', 
    'caseversion': np.int32, 
    'i_f_code': 'object', 
    'event_dt': 'object', 
    'mfr_dt': 'object',
    'init_fda_dt': 'object', 
    'fda_dt': 'object', 
    'rept_cod': 'object', 
    'auth_num': 'object', 
    'mfr_num': 'object', 
    'mfr_sndr': 'object',
    'lit_ref': 'object', 
    'age': np.float64, 
    'age_cod': 'object', 
    'age_grp': 'object', 
    'sex': 'object', 
    'e_sub': 'object', 
    'wt': np.float64, 
    'wt_cod': 'object',
    'rept_dt': 'object', 
    'to_mfr': 'object', 
    'occp_cod': 'object', 
    'reporter_country': 'object', 
    'occr_country': 'object'
}

# {‘a’: np.float64, ‘b’: np.int32, ‘c’: ‘Int64’}

In [23]:
demo = pd.read_csv(ascii_file_demo, sep='$', dtype=datatypes)

In [24]:
demo.columns

Index(['primaryid', 'caseid', 'caseversion', 'i_f_code', 'event_dt', 'mfr_dt',
       'init_fda_dt', 'fda_dt', 'rept_cod', 'auth_num', 'mfr_num', 'mfr_sndr',
       'lit_ref', 'age', 'age_cod', 'age_grp', 'sex', 'e_sub', 'wt', 'wt_cod',
       'rept_dt', 'to_mfr', 'occp_cod', 'reporter_country', 'occr_country'],
      dtype='object')

In [26]:
demo.head()

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
0,100035916,10003591,6,F,20130718,20181203,20140312,20181211,EXP,NaN,PHHY2013GB101660,NOVARTIS,NaN,47.00,YR,NaN,F,Y,nan,NaN,20181211,NaN,OT,GB,GB
1,100050413,10005041,3,F,20140306,20141118,20140312,20181213,EXP,NaN,US-TEVA-468475USA,TEVA,NaN,25.00,YR,NaN,F,Y,68.10,KG,20181213,NaN,CN,US,US
2,1000551312,10005513,12,F,20120209,20181107,20140313,20181115,EXP,NaN,BR-AMGEN-BRASP2012013548,AMGEN,NaN,55.00,YR,A,F,Y,67.00,KG,20181115,NaN,CN,BR,BR
3,100058832,10005883,2,F,NaN,20180928,20140313,20181012,EXP,NaN,FR-RANBAXY-2014RR-78735,RANBAXY,NaN,31.00,YR,NaN,F,Y,nan,NaN,20181012,NaN,OT,GB,FR
4,100065479,10006547,9,F,201203,20181211,20140313,20181228,EXP,NaN,US-BAYER-2014-035909,BAYER,NaN,36.00,YR,A,F,Y,90.70,KG,20181228,NaN,CN,US,US


In [27]:
demo.describe(include='all')

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
count,394066,394066,394066.00,394066,205438,370593,394066,394066,394066,20168,370595,394065,23441,235444.00,235452,80189,347760,394066,81142.00,81142,393749,23479,387070,394066,394053
unique,394066,394066,nan,2,4711,2370,2503,183,3,15597,370595,471,17759,nan,6,6,3,2,nan,2,351,3,5,160,163
top,154544191,14508418,nan,I,2018,20181210,20181016,20181016,EXP,0000,PHJP2018JP021151,PFIZER,"STACEY R, VERA T, MORGAN T, JORDAN J, WHITLOCK...",nan,YR,A,F,Y,nan,KG,20181016,N,CN,US,US
freq,1,1,nan,267661,25293,6857,11177,12657,204438,14,1,35409,79,nan,230226,48200,212580,370587,nan,80809,11312,22175,168973,249968,262062
mean,NaN,NaN,1.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200.03,NaN,NaN,NaN,NaN,75.17,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1843.75,NaN,NaN,NaN,NaN,29.24,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-10.00,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,45.00,NaN,NaN,NaN,NaN,59.87,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60.00,NaN,NaN,NaN,NaN,72.58,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.00,NaN,NaN,NaN,NaN,88.45,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 394066 entries, 0 to 394065
Data columns (total 25 columns):
primaryid           394066 non-null object
caseid              394066 non-null object
caseversion         394066 non-null int32
i_f_code            394066 non-null object
event_dt            205438 non-null object
mfr_dt              370593 non-null object
init_fda_dt         394066 non-null object
fda_dt              394066 non-null object
rept_cod            394066 non-null object
auth_num            20168 non-null object
mfr_num             370595 non-null object
mfr_sndr            394065 non-null object
lit_ref             23441 non-null object
age                 235444 non-null float64
age_cod             235452 non-null object
age_grp             80189 non-null object
sex                 347760 non-null object
e_sub               394066 non-null object
wt                  81142 non-null float64
wt_cod              81142 non-null object
rept_dt             393749 non-nu

The ASCIIs look easier to work with, and according to the documentation, most of the information they contain should be the same, although both file types contain some extra fields and miss some other fields.  

We'll proceed with the ASCII files first, and add any supplemental info from XMLs later if needed.

##### DEMO file contents  

From above, the number of case reports in the 2018Q4 DEMO file is 394,066, which is consistent with the number supplied by FDA in the accompanying documentation.

##### Unique identifiers

The `primaryid` field is the unique identifier for a current case report in the data, and it is a combination of `caseid` and `caseversion`.

In [37]:
demo.primaryid.describe()

count        394066
unique       394066
top       154544191
freq              1
Name: primaryid, dtype: object

The unique record identifier is indeed unique. Great.

In [38]:
demo.caseid.describe()

count       394066
unique      394066
top       14508418
freq             1
Name: caseid, dtype: object

In [39]:
demo.caseversion.describe()

count   394066.00
mean         1.67
std          1.75
min          1.00
25%          1.00
50%          1.00
75%          2.00
max         88.00
Name: caseversion, dtype: float64

In [43]:
demo[["primaryid", "caseid", "caseversion"]].head()

,primaryid,caseid,caseversion
0,100035916,10003591,6
1,100050413,10005041,3
2,1000551312,10005513,12
3,100058832,10005883,2
4,100065479,10006547,9


##### Distribution of case version values

In [313]:
show_value_counts(demo.caseversion)

,caseversion,count,proportion
0,1,267661,0.68
1,2,74684,0.19
2,3,25090,0.06
3,4,11270,0.03
4,5,5559,0.01
5,6,3129,0.01
6,7,1857,0.00
7,8,1261,0.00
8,9,824,0.00
9,10,570,0.00


The majority of most recent case version numbers are 1 (68%), 2 (19%) and 3 (6%), accounting for 93% of the cases. About 2% of all cases have most recent case version number that are above 6. The highest case version number is 88. No missing values. 

##### i_f_code  

From documentation:
> Code for initial or follow-up status of report, as reported
by manufacturer.
>
> | CODE | MEANING_TEXT |
| ---- |------------- |
| I    | Initial      |
| F    | Follow-up    |


In [110]:
demo.i_f_code.describe()

count     394066
unique         2
top            I
freq      267661
Name: i_f_code, dtype: object

In [314]:
show_value_counts(demo.i_f_code)

,i_f_code,count,proportion
0,I,267661,0.68
1,F,126405,0.32


This is consistent with the 68% of records with caseversion=1 shown above. No missing values in this field.

##### event_dt

From documentation:  

> Date the adverse event occurred or began. (YYYYMMDD format) –
If a complete date is not available, a partial date is
provided.

In [111]:
demo.event_dt.describe()

count     205438
unique      4711
top         2018
freq       25293
Name: event_dt, dtype: object

In [315]:
show_value_counts(demo.event_dt, 20)

,event_dt,count,proportion
0,NaN,188628,0.48
1,2018,25293,0.06
2,201810,4210,0.01
3,201809,4096,0.01
4,2017,4057,0.01
5,201808,3189,0.01
6,201811,2739,0.01
7,2016,2353,0.01
8,201807,2290,0.01
9,2015,1785,0.00


Nearly half of the adverse event cases do not have a date for when the adverse event occurred or began. The missing values count is consistent with the number provided by the FDA.

##### mfr_dt  

From documentation:  

> Date manufacturer first received initial information. In
subsequent   versions of a case, the latest manufacturer
received date will be   provided (YYYYMMDD format). If a
complete date is not available, a   partial date will be
provided.

In [118]:
demo.mfr_dt.describe()

count       370593
unique        2370
top       20181210
freq          6857
Name: mfr_dt, dtype: object

In [316]:
show_value_counts(demo.mfr_dt, 20)

,mfr_dt,count,proportion
0,NaN,23473,0.06
1,20181210,6857,0.02
2,20181203,5742,0.01
3,20181029,5662,0.01
4,20181126,5605,0.01
5,20181211,5482,0.01
6,20181001,5481,0.01
7,20181009,5465,0.01
8,20181217,5312,0.01
9,20181022,5298,0.01


There are 6% missing values for this field. The missing values count is consistent with the FDA number.

##### init_fda_dt  

From documentation:

> Date FDA received first version (Initial) of Case (YYYYMMDD format)

In [121]:
demo.init_fda_dt.describe()

count       394066
unique        2503
top       20181016
freq         11177
Name: init_fda_dt, dtype: object

In [318]:
show_value_counts(demo.init_fda_dt, 20)

,init_fda_dt,count,proportion
0,20181016,11177,0.03
1,20181017,7905,0.02
2,20181018,6215,0.02
3,20181217,6142,0.02
4,20181120,6061,0.02
5,20181129,5967,0.02
6,20181015,5741,0.01
7,20181010,5285,0.01
8,20181116,5029,0.01
9,20181102,5020,0.01


No missing values.

##### fda_dt  

From documentation:  

> Date FDA received Case. In subsequent versions of a case, the latest manufacturer received date will be provided (YYYYMMDD format).

In [125]:
demo.fda_dt.describe()

count       394066
unique         183
top       20181016
freq         12657
Name: fda_dt, dtype: object

In [319]:
show_value_counts(demo.fda_dt, 20)

,fda_dt,count,proportion
0,20181016,12657,0.03
1,20181017,9620,0.02
2,20181217,8111,0.02
3,20181129,7432,0.02
4,20181120,7355,0.02
5,20181018,7265,0.02
6,20181227,6866,0.02
7,20181221,6863,0.02
8,20181015,6783,0.02
9,20181228,6514,0.02


No missing values, consistent with FDA number.

##### rept_cod  

From documentation:  

> Code for the type of report submitted (See table below)
> 
> | CODE | MEANING_TEXT
| ---- | ---------------
| EXP  | Expedited (15-Day)
| PER  | Periodic (Non-Expedited)
| DIR  | Direct
>
> Expedited (15-day) and Periodic (Non-Expedited) reports are from manufacturers; "Direct" reports are voluntarily
submitted to the FDA by non-manufacturers.





In [129]:
demo.rept_cod.describe()

count     394066
unique         3
top          EXP
freq      204438
Name: rept_cod, dtype: object

In [320]:
show_value_counts(demo.rept_cod)

,rept_cod,count,proportion
0,EXP,204438,0.52
1,PER,166157,0.42
2,DIR,23471,0.06


No missing values.

##### auth_num  

From documentation:  

> Regulatory Authority’s case report number, when available.  
> \* New tag added in 2014Q3 extract.

In [132]:
demo.auth_num.describe()

count     20168
unique    15597
top        0000
freq         14
Name: auth_num, dtype: object

In [321]:
show_value_counts(demo.auth_num, 10)

,auth_num,count,proportion
0,NaN,373898,0.95
1,0000,14,0.00
2,00,11,0.00
3,DE-CADRBFARM-2018025631,10,0.00
4,GB-MHRA-EYC 00190348,9,0.00
5,FR-AFSSAPS-TS20180923,8,0.00
6,FR-AFSSAPS-CN20182166,8,0.00
7,GB-MHRA-EYC 00188736,7,0.00
8,GB-MHRA-ADR 22496422,7,0.00
9,FR-AFSSAPS-AM20180734,7,0.00


Mostly missing values, with a couple of other values (like `00` ) that may be placeholders/defaults for missing values. 

##### mfr_num  

From documentation:  

> Manufacturer's unique report identifier.

In [135]:
demo.mfr_num.describe()

count               370595
unique              370595
top       PHJP2018JP021151
freq                     1
Name: mfr_num, dtype: object

In [322]:
show_value_counts(demo.mfr_num, 10)

,mfr_num,count,proportion
0,NaN,23471,0.06
1,US-ROCHE-2112065,1,0.00
2,PHHO2018CA011677,1,0.00
3,CL-PROVELL PHARMACEUTICALS-2056832,1,0.00
4,US-IGSA-SR10006388,1,0.00
5,CN-ROCHE-2208075,1,0.00
6,"PH-B.I. PHARMACEUTICALS,INC./RIDGEFIELD-2018-B...",1,0.00
7,US-AMGEN-USASP2018185967,1,0.00
8,CA-ROCHE-2190755,1,0.00
9,PHHY2016IT042284,1,0.00


6% of values are missing, and the missing values count is consistent with the FDA number. The non-missing values are unique, as expected.

##### mfr_sndr  

From documentation:  

> Coded name of manufacturer sending report; if not found, then verbatim name of organization sending report.

In [139]:
demo.mfr_sndr.describe()

count     394065
unique       471
top       PFIZER
freq       35409
Name: mfr_sndr, dtype: object

In [323]:
show_value_counts(demo.mfr_sndr, 20)

,mfr_sndr,count,proportion
0,PFIZER,35409,0.09
1,AMGEN,30828,0.08
2,NOVARTIS,25360,0.06
3,FDA-CTU,23470,0.06
4,SANOFI AVENTIS,18107,0.05
5,JANSSEN,14866,0.04
6,CELGENE,13511,0.03
7,BRISTOL MYERS SQUIBB,13442,0.03
8,TEVA,12871,0.03
9,ABBVIE,11719,0.03


In [155]:
# count missing values
demo.mfr_sndr.isna().sum()

1

In [156]:
demo[demo.mfr_sndr.isna()]

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
203873,155751552,15575155,2,F,NaN,NaN,20181026,20181026,DIR,NaN,NaN,NaN,NaN,71.00,YR,NaN,M,N,nan,NaN,20181025,N,OT,US,US


One missing value, consistent with the FDA number.

##### lit_ref  

From documentation:  

> Literature Reference information, when available; populated with last 500 characters if >500 characters are available.
>
> \* New tag added in 2014Q3 extract.

In [157]:
demo.lit_ref.describe()

count                                                 23441
unique                                                17759
top       STACEY R, VERA T, MORGAN T, JORDAN J, WHITLOCK...
freq                                                     79
Name: lit_ref, dtype: object

In [324]:
show_value_counts(demo.lit_ref, 10)

,lit_ref,count,proportion
0,NaN,370625,0.94
1,"STACEY R, VERA T, MORGAN T, JORDAN J, WHITLOCK...",79,0.00
2,"DOI: 10.4081/NI.2018.7469#. LAPMAG A, LERTSINU...",71,0.00
3,"GLEESON M, PECKITT C, TO YM, EDWARDS L, OATES ...",70,0.00
4,NOT APPLICABLE,61,0.00
5,"BISHOP-FREEMAN SC, HENSEL EM, FEASTER MS, WINE...",53,0.00
6,"GUMMIN, D.. 2016 ANNUAL REPORT OF THE AMERICAN...",48,0.00
7,"STRUGOV V, STADNIK E, VIRTS Y, ANDREEVA T, ZAR...",45,0.00
8,"DALKILIC E, COSKUN BN, YAGIZ B, TUFAN AN, ERMU...",40,0.00
9,"JABEEN SA, GADDAMANUGU P, CHERIAN A, MRIDULA K...",38,0.00


94% of the values are missing, and 61 records have this value set to "NOT APPLICABLE".

##### age  

From documentation:  

> Numeric value of patient's age at event.

In [162]:
demo.age.describe()

count   235444.00
mean       200.03
std       1843.75
min        -10.00
25%         45.00
50%         60.00
75%         71.00
max      34926.00
Name: age, dtype: float64

In [325]:
show_value_counts(demo.age)

,age,count,proportion
0,nan,158622,0.40
1,70.00,5562,0.01
2,65.00,5499,0.01
3,63.00,5440,0.01
4,60.00,5394,0.01
5,68.00,5333,0.01
6,64.00,5316,0.01
7,62.00,5312,0.01
8,67.00,5191,0.01
9,69.00,5178,0.01


In [183]:
demo[demo.age < 0]

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
263592,156388221,15638822,1,I,20180429,20180508,20181120,20181120,PER,NaN,US-PERRIGO-18US005100,PERRIGO,NaN,-10.00,YR,NaN,F,Y,77.98,KG,20181120,NaN,CN,US,US


In [188]:
len(demo[demo.age > 100])

2011

In [189]:
demo[demo.age > 100].head()

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
532,104248582,10424858,2,F,201407,20181107,20140902,20181115,EXP,NaN,US-ASTRAZENECA-2014SE63425,ASTRAZENECA,NaN,552.00,MON,NaN,M,Y,79.40,KG,20181115,NaN,NaN,US,US
655,1050904010,10509040,10,F,2014,20181030,20141009,20181107,EXP,NaN,US-ASTRAZENECA-2014SE69488,ASTRAZENECA,NaN,801.00,MON,NaN,F,Y,50.30,KG,20181107,NaN,NaN,US,US
1075,107543802,10754380,2,F,201501,20181031,20150202,20181112,PER,NaN,US-ASTRAZENECA-2015SE07497,ASTRAZENECA,NaN,1023.00,MON,NaN,M,Y,65.80,KG,20181112,NaN,NaN,US,US
1211,108765514,10876551,4,F,20130401,20180919,20150301,20181019,PER,NaN,US-ASTRAZENECA-2013SE23016,ASTRAZENECA,NaN,25245.00,DY,NaN,F,Y,101.20,KG,20181019,NaN,NaN,US,US
1337,109705173,10970517,3,F,201411,20181119,20150331,20181122,EXP,NaN,US-ASTRAZENECA-2015SE28983,ASTRAZENECA,NaN,764.00,MON,NaN,F,Y,93.00,KG,20181122,NaN,NaN,US,US


Age is missing in 40% of the records. There is one record with a negative age value, which will need to be cleaned. Most of the greater than 100 age values are coded in some other increment than a year, e.g. a month or a day.  

The missing values count is consistent with the FDA number.

##### age_cod  

From documentation:  

> Unit abbreviation for patient's age (See table below)  
>
> | CODE      | MEANING_TEXT
| ----      | ------------
| DEC       | DECADE
| YR        | YEAR
| MON       | MONTH
| WK        | WEEK
| DY        | DAY
| HR        | HOUR

In [190]:
demo.age_cod.describe()

count     235452
unique         6
top           YR
freq      230226
Name: age_cod, dtype: object

In [326]:
show_value_counts(demo.age_cod)

,age_cod,count,proportion
0,YR,230226,0.58
1,NaN,158614,0.40
2,DY,1935,0.00
3,DEC,1618,0.00
4,MON,1536,0.00
5,WK,127,0.00
6,HR,10,0.00


In [207]:
demo[(demo.age.isna()) & (demo.age_cod.notna())]

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
119358,154870241,15487024,1,I,NaN,NaN,20181010,20181010,DIR,NaN,NaN,FDA-CTU,NaN,nan,YR,NaN,F,N,20.87,KG,20181010,N,CN,US,US
200400,155714611,15571461,1,I,20181016,NaN,20181030,20181030,DIR,NaN,NaN,FDA-CTU,NaN,nan,WK,NaN,M,N,3.13,KG,20181030,N,MD,US,US
234374,156076701,15607670,1,I,NaN,NaN,20181112,20181112,DIR,NaN,NaN,FDA-CTU,NaN,nan,YR,NaN,F,N,nan,NaN,20181112,N,NaN,US,US
281552,156581371,15658137,1,I,NaN,NaN,20181115,20181115,DIR,NaN,NaN,FDA-CTU,NaN,nan,DY,NaN,F,N,54.43,KG,20181114,N,OT,US,US
318373,156982451,15698245,1,I,20181109,NaN,20181126,20181126,DIR,NaN,NaN,FDA-CTU,NaN,nan,YR,NaN,M,N,nan,NaN,20181120,N,OT,US,US
322785,157031521,15703152,1,I,20180918,NaN,20181130,20181130,DIR,NaN,NaN,FDA-CTU,NaN,nan,YR,NaN,M,N,nan,NaN,20181130,N,PH,US,US
332043,157133731,15713373,1,I,20180524,NaN,20181129,20181129,DIR,NaN,NaN,FDA-CTU,NaN,nan,YR,NaN,NaN,N,85.55,KG,20180822,N,PH,US,US
382724,157711891,15771189,1,I,NaN,NaN,20181227,20181227,DIR,NaN,NaN,FDA-CTU,NaN,nan,YR,NaN,F,N,11.00,KG,20181227,N,NaN,US,US


In [208]:
len(demo[(demo.age.isna()) & (demo.age_cod.notna())])

8

In [209]:
demo[(demo.age.notna()) & (demo.age_cod.isna())]

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country


This field is missing in 40% of the records, which matches the 40% of the records with missing age. Of the non-missing values, most of them are in years.  

The missing values count is consistent with the FDA number.

##### age_grp  

From documentation:  

> Patient Age Group code as follows, when available:
>
> | CODE   | MEANING_TEXT
| ----   | ------------
|  N     |  Neonate
|  I     |  Infant
|  C     |  Child
|  T     |  Adolescent
|  A     |  Adult
|  E     |  Elderly
>
> \* New tag added in 2014Q3 extract.

In [210]:
demo.age_grp.describe()

count     80189
unique        6
top           A
freq      48200
Name: age_grp, dtype: object

In [327]:
show_value_counts(demo.age_grp)

,age_grp,count,proportion
0,NaN,313877,0.80
1,A,48200,0.12
2,E,27869,0.07
3,C,1547,0.00
4,T,1129,0.00
5,N,916,0.00
6,I,528,0.00


80% of the values are missing, compared to the 40% missing age values. The missing value counts are consistent with the FDA number.

##### sex  

From documentation:  

> Code for patient's sex (See table below)  
>
> | CODE      | MEANING_TEXT
| ----      | ------------
| UNK       | Unknown
| M         | Male
| F | Female

In [213]:
demo.sex.describe()

count     347760
unique         3
top            F
freq      212580
Name: sex, dtype: object

In [328]:
show_value_counts(demo.sex)

,sex,count,proportion
0,F,212580,0.54
1,M,135150,0.34
2,NaN,46306,0.12
3,UNK,30,0.00


12% missing values. The frequency counts and percentages are consistent with the FDA numbers.

##### e_sub  

From documentation:  

> Whether (Y/N) this report was submitted under the electronic submissions procedure for manufacturers.

In [216]:
demo.e_sub.describe()

count     394066
unique         2
top            Y
freq      370587
Name: e_sub, dtype: object

In [329]:
show_value_counts(demo.e_sub)

,e_sub,count,proportion
0,Y,370587,0.94
1,N,23479,0.06


No missing values. The frequency counts are consistent with the FDA numbers.

##### wt  

From documentation:  

> Numeric value of patient's weight.

In [220]:
demo.wt.describe()

count   81142.00
mean       75.17
std        29.24
min         0.00
25%        59.87
50%        72.58
75%        88.45
max      2890.00
Name: wt, dtype: float64

In [330]:
show_value_counts(demo.wt)

,wt,count,proportion
0,nan,312924,0.79
1,70.00,1404,0.00
2,60.00,1343,0.00
3,65.00,1094,0.00
4,68.00,1039,0.00
5,80.00,990,0.00
6,75.00,949,0.00
7,90.00,844,0.00
8,63.00,834,0.00
9,72.00,803,0.00


79% missing values. Missing value counts are consistent with the FDA number.

##### wt_cod  

From documentation:  

> Unit abbreviation for patient's weight (See table below)  
>
> | CODE     | MEANING_TEXT
| ----     | ------------
| KG       |  Kilograms
| LBS      |  Pounds
| GMS |  Grams

In [223]:
demo.wt_cod.describe()

count     81142
unique        2
top          KG
freq      80809
Name: wt_cod, dtype: object

In [331]:
show_value_counts(demo.wt_cod)

,wt_cod,count,proportion
0,NaN,312924,0.79
1,KG,80809,0.21
2,LBS,333,0.00


79% missing values, consistent with the 79% missing weight values.  
Missing value counts are consistent with the FDA number.

##### rept_dt  

From documentation:  

> Date report was sent (YYYYMMDD format). If a complete date is not available, a partial date is provided. 

In [226]:
demo.rept_dt.describe()

count       393749
unique         351
top       20181016
freq         11312
Name: rept_dt, dtype: object

In [332]:
show_value_counts(demo.rept_dt, 20)

,rept_dt,count,proportion
0,20181016,11312,0.03
1,20181017,9311,0.02
2,20181015,7918,0.02
3,20181018,7689,0.02
4,20181120,7604,0.02
5,20181217,7069,0.02
6,20181129,7004,0.02
7,20181227,6808,0.02
8,20181218,6571,0.02
9,20181219,6562,0.02


In [230]:
# missing values count
demo.rept_dt.isna().sum()

317

In [232]:
demo.rept_dt.isna().sum()/demo.primaryid.count()

0.0008044337750529099

In [233]:
demo[demo.rept_dt.isna()].head(20)

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
20427,142482596,14248259,6,F,2017,20181023,20171204,20181029,EXP,FR-002147023-PHHY2017FR176929,PHHY2017FR176929,NOVARTIS,NaN,78.00,YR,NaN,F,Y,80.00,KG,NaN,NaN,OT,FR,FR
42888,150317284,15031728,4,F,20170306,20180722,20180619,20180727,EXP,NaN,PHHY2018FR024978,NOVARTIS,NaN,68.00,YR,NaN,M,Y,96.00,KG,NaN,NaN,OT,FR,FR
54684,152340061,15234006,1,I,NaN,20180606,20180802,20180802,EXP,NaN,PHHY2018ES063808,SANDOZ,NaN,nan,NaN,A,F,Y,nan,NaN,NaN,NaN,OT,ES,ES
54685,152340091,15234009,1,I,20180217,20180601,20180802,20180802,EXP,NaN,PHHY2018GB063683,SANDOZ,NaN,51.00,YR,NaN,M,Y,82.55,KG,NaN,NaN,OT,GB,GB
54707,152341871,15234187,1,I,20160123,20180724,20180802,20180802,EXP,FR-AFSSAPS-ST20181059,FR-TEVA-2018-FR-932970,TEVA,NaN,75.00,YR,NaN,F,Y,nan,NaN,NaN,NaN,MD,FR,FR
63135,153133691,15313369,1,I,201807,20180814,20180823,20180823,PER,NaN,US-TEVA-2018-US-945214,TEVA,NaN,nan,NaN,NaN,F,Y,nan,NaN,NaN,NaN,CN,US,US
64324,153236361,15323636,1,I,20180806,20180817,20180827,20180827,EXP,NaN,PHHY2018DE078420,SANDOZ,NaN,29.00,YR,NaN,M,Y,nan,NaN,NaN,NaN,CN,DE,DE
66091,153364721,15336472,1,I,20180623,20180819,20180830,20180830,EXP,NaN,PHHY2018FR081625,SANDOZ,NaN,73.00,YR,NaN,F,Y,82.00,KG,NaN,NaN,OT,FR,FR
83914,154488013,15448801,3,F,199606,20180925,20180929,20180929,EXP,NaN,PHHY2018AT111525,NOVARTIS,NaN,67.00,YR,NaN,M,Y,nan,NaN,NaN,NaN,OT,AT,AT
92411,154587971,15458797,1,I,NaN,NaN,20181001,20181001,DIR,NaN,NaN,FDA-CTU,NaN,64.00,YR,NaN,F,N,89.81,KG,NaN,N,CN,US,US


Less than 1% of missing values. Missing values count is consistent with the FDA number.

##### to_mfr  

From documentation:  

> Whether (Y/N) voluntary reporter also notified manufacturer (blank for manufacturer reports).

In [234]:
demo.to_mfr.describe()

count     23479
unique        3
top           N
freq      22175
Name: to_mfr, dtype: object

In [333]:
show_value_counts(demo.to_mfr)

,to_mfr,count,proportion
0,NaN,370587,0.94
1,N,22175,0.06
2,Y,1303,0.00
3,U,1,0.00


In [237]:
demo[demo.to_mfr == "U"]

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
390524,158228921,15822892,1,I,20181204,NaN,20181217,20181217,DIR,NaN,NaN,FDA-CTU,NaN,1.00,DY,NaN,M,N,nan,NaN,20181213,U,OT,US,US


94% of the values are missing. The Y, N and missing value counts match the FDA numbers, but the U value with a count of 1 is not present in the FDA numbers in the accompanying pdf file. The record with this value is displayed above.

##### occp_cod  

From documentation:  

> Abbreviation for the reporter's type of occupation in the latest version of a case.
>
> | CODE      | MEANING_TEXT
| ----      | ------------
| MD        | Physician
| PH        | Pharmacist
| OT        | Other health-professional
| LW        | Lawyer
| CN | Consumer

In [238]:
demo.occp_cod.describe()

count     387070
unique         5
top           CN
freq      168973
Name: occp_cod, dtype: object

In [334]:
show_value_counts(demo.occp_cod)

,occp_cod,count,proportion
0,CN,168973,0.43
1,MD,99246,0.25
2,OT,80654,0.20
3,PH,31766,0.08
4,NaN,6996,0.02
5,LW,6431,0.02


There are 2% missing values. The frequency counts are consistent with the FDA numbers.

##### reporter_country  

ISO country codes can be found here: https://www.iso.org/obp/ui/#search/code/  


From documentation:  

> The country of the reporter in the latest version of a case.
>
> \* Note: the links to the country codes in the documentation don't really work.

   

In [241]:
demo.reporter_country.describe()

count     394066
unique       160
top           US
freq      249968
Name: reporter_country, dtype: object

In [335]:
show_value_counts(demo.reporter_country, 20)

,reporter_country,count,proportion
0,US,249968,0.63
1,CA,16897,0.04
2,GB,16739,0.04
3,FR,15736,0.04
4,JP,15711,0.04
5,COUNTRY NOT SPECIFIED,14719,0.04
6,DE,10502,0.03
7,IT,7833,0.02
8,ES,4681,0.01
9,BR,3454,0.01


There are no missing values, which is consistent with the FDA number.  
However, about 4% of the case records have "COUNTRY NOT SPECIFIED" in this field.

##### occr_country  

_From documentation:_  
> The country where the event occurred.

In [244]:
demo.occr_country.describe()

count     394053
unique       163
top           US
freq      262062
Name: occr_country, dtype: object

In [336]:
show_value_counts(demo.occr_country, 20)

,occr_country,count,proportion
0,US,262062,0.67
1,CA,17550,0.04
2,FR,16969,0.04
3,JP,15968,0.04
4,GB,13561,0.03
5,DE,10752,0.03
6,IT,8315,0.02
7,ES,4896,0.01
8,BR,3873,0.01
9,CN,2988,0.01


In [247]:
# missing values count
demo.occr_country.isna().sum()

13

In [254]:
demo.occr_country.str.len().value_counts(dropna=False)

2.00    394053
nan         13
Name: occr_country, dtype: int64

There is less than 1% missing values. The missing values count is consistent with the FDA number.

##### Summary for DEMO ASCII file  

The data is mostly consistent with the accompanying FDA missing value and frequency counts pdf. 
Some fields have low counts of non-missing values, which can be problematic for analyses.  

* **Data quality issues found:**  
  - one record has a negative value in the `age` field  
  - the field `lit_ref` has the value "NOT APPLICABLE" in 61 records, in addition to the null missing values.
  - one record has a "U" categorical value in the `to_mfr` field, which wasn't listed in the FDA pdf  
  - While the `reporter_country` field does not have null missing values, it does contain the value "COUNTRY NOT SPECIFIED", which indicates missing country values. About 4% of records have this value.   
  
  
* **Data cleaning steps to do:**  
  - fix the data quality issues listed above
  - standardize weight and age fields to SI units  
  - infer age categories

#### 4.2.2 DRUG file

In [255]:
!head data/raw/ascii_2018q4/ascii/DRUG18Q4.txt

primaryid$caseid$drug_seq$role_cod$drugname$prod_ai$val_vbm$route$dose_vbm$cum_dose_chr$cum_dose_unit$dechal$rechal$lot_num$exp_dt$nda_num$dose_amt$dose_unit$dose_form$dose_freq
100035916$10003591$1$PS$GILENYA$FINGOLIMOD HYDROCHLORIDE$1$Oral$QD$$$$$$$022527$$$CAPSULE$QD
100050413$10005041$1$PS$PLAN B ONE-STEP$LEVONORGESTREL$1$Oral$1.5 MILLIGRAM DAILY;$$$D$$$$021998$1.5$MG$TABLET$QD
1000551312$10005513$1$PS$ENBREL$ETANERCEPT$1$Subcutaneous$50 MG, ONCE WEEKLY$50$MG$U$$ G79072$$103795$50$MG$SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE$/wk
1000551312$10005513$2$SS$ENBREL$ETANERCEPT$1$Unknown$50 MG, ONCE WEEKLY (EVERY THURSDAY)$50$MG$U$$ S77448$$103795$50$MG$SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE$/wk
1000551312$10005513$3$SS$ENBREL$ETANERCEPT$1$Unknown$1 DF, WEEKLY$50$MG$U$$$$103795$1$DF$SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE$/wk
1000551312$10005513$4$SS$ENBREL$ETANERCEPT$1$Unknown$UNK$50$MG$U$$$$103795$$$SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE$
1000551312$10005513$5$SS$ENB

In [256]:
ascii_file_drug = raw_ascii_path + "DRUG18Q4.txt"

In [275]:
datatypes = {
    'primaryid': 'object', 
    'caseid': 'object', 
    'drug_seq': np.int32, 
    'role_cod': 'object', 
    'drugname': 'object', 
    'prod_ai': 'object',
    'val_vbm': 'object', 
    'route': 'object', 
    'dose_vbm': 'object', 
    'cum_dose_chr': np.float64, 
    'cum_dose_unit': 'object',
    'dechal': 'object', 
    'rechal': 'object', 
    'lot_num': 'object', 
    'exp_dt': 'object', 
    'nda_num': 'object', 
    'dose_amt': np.float64,
    'dose_unit': 'object', 
    'dose_form': 'object', 
    'dose_freq': 'object'
}

In [270]:
drug = pd.read_csv(ascii_file_drug, sep='$', dtype=datatypes)

In [271]:
drug.columns

Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai',
       'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit',
       'dechal', 'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt',
       'dose_unit', 'dose_form', 'dose_freq'],
      dtype='object')

In [272]:
drug.head()

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
0,100035916,10003591,1,PS,GILENYA,FINGOLIMOD HYDROCHLORIDE,1,Oral,QD,nan,NaN,NaN,NaN,NaN,NaN,022527,nan,NaN,CAPSULE,QD
1,100050413,10005041,1,PS,PLAN B ONE-STEP,LEVONORGESTREL,1,Oral,1.5 MILLIGRAM DAILY;,nan,NaN,D,NaN,NaN,NaN,021998,1.50,MG,TABLET,QD
2,1000551312,10005513,1,PS,ENBREL,ETANERCEPT,1,Subcutaneous,"50 MG, ONCE WEEKLY",50.00,MG,U,NaN,G79072,NaN,103795,50.00,MG,SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE,/wk
3,1000551312,10005513,2,SS,ENBREL,ETANERCEPT,1,Unknown,"50 MG, ONCE WEEKLY (EVERY THURSDAY)",50.00,MG,U,NaN,S77448,NaN,103795,50.00,MG,SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE,/wk
4,1000551312,10005513,3,SS,ENBREL,ETANERCEPT,1,Unknown,"1 DF, WEEKLY",50.00,MG,U,NaN,NaN,NaN,103795,1.00,DF,SOLUTION FOR INJECTION IN PRE-FILLED SYRINGE,/wk


In [273]:
drug.describe(include='all')

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
count,1546835,1546835,1546835.00,1546835,1546823,1511985,1546835,1102380,915572,49001.00,49011,794509,264528,236897,4715,499438,650429.00,650429,672867,410843
unique,394066,394066,nan,4,61846,5782,2,66,143318,nan,23,4,4,42141,716,6385,nan,36,372,33
top,146088398,14608839,nan,C,REVLIMID,ASPIRIN,1,Unknown,UNK,nan,MG,U,U,UNKNOWN,20200131,021880,nan,MG,TABLET,QD
freq,310,310,nan,714731,15475,19377,1511988,463084,278054,nan,35054,431577,226288,77892,193,14551,nan,506064,176663,229695
mean,NaN,NaN,7.20,NaN,NaN,NaN,NaN,NaN,NaN,58156.82,NaN,NaN,NaN,NaN,NaN,NaN,598.74,NaN,NaN,NaN
std,NaN,NaN,11.60,NaN,NaN,NaN,NaN,NaN,NaN,5321159.75,NaN,NaN,NaN,NaN,NaN,NaN,85133.95,NaN,NaN,NaN
min,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
25%,NaN,NaN,1.00,NaN,NaN,NaN,NaN,NaN,NaN,80.00,NaN,NaN,NaN,NaN,NaN,NaN,5.00,NaN,NaN,NaN
50%,NaN,NaN,4.00,NaN,NaN,NaN,NaN,NaN,NaN,610.00,NaN,NaN,NaN,NaN,NaN,NaN,30.00,NaN,NaN,NaN
75%,NaN,NaN,8.00,NaN,NaN,NaN,NaN,NaN,NaN,5100.00,NaN,NaN,NaN,NaN,NaN,NaN,150.00,NaN,NaN,NaN


In [274]:
drug.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1546835 entries, 0 to 1546834
Data columns (total 20 columns):
primaryid        1546835 non-null object
caseid           1546835 non-null object
drug_seq         1546835 non-null int32
role_cod         1546835 non-null object
drugname         1546823 non-null object
prod_ai          1511985 non-null object
val_vbm          1546835 non-null object
route            1102380 non-null object
dose_vbm         915572 non-null object
cum_dose_chr     49001 non-null float64
cum_dose_unit    49011 non-null object
dechal           794509 non-null object
rechal           264528 non-null object
lot_num          236897 non-null object
exp_dt           4715 non-null object
nda_num          499438 non-null object
dose_amt         650429 non-null float64
dose_unit        650429 non-null object
dose_form        672867 non-null object
dose_freq        410843 non-null object
dtypes: float64(2), int32(1), object(17)
memory usage: 230.1+ MB


##### Unique identifyers

The datarows in the DRUG file are unique by `primaryid` + `drug_seq`.  
The DRUG file has a many-to-one relationship with the DEMO file, matching on `primaryid`.  
The DRUG file also has the `caseid` field. Both  `primaryid` and `caseid` fields here are defined the same way as in the DEMO file, and across the rest of the datasets.

In [278]:
drug.primaryid.describe()

count       1546835
unique       394066
top       146088398
freq            310
Name: primaryid, dtype: object

In [337]:
show_value_counts(drug.primaryid, 20)

,primaryid,count,proportion
0,146088398,310,0.00
1,153490012,259,0.00
2,156811382,256,0.00
3,148011912,252,0.00
4,153486263,227,0.00
5,156232931,200,0.00
6,153535922,200,0.00
7,156196031,198,0.00
8,155573181,197,0.00
9,155787136,195,0.00


The number of unique values of `primaryid` matches the number of records in the DEMO files. 

In [283]:
drug.caseid.describe()

count      1546835
unique      394066
top       14608839
freq           310
Name: caseid, dtype: object

Same as above, the number of unique values of `caseid` matches the number of records in the DEMO files. 

##### drug_seq  

From documentation:

> Unique number for identifying a drug for a Case.  
> To link to the THERyyQq.TXT data file, both the Case number (primary key) and the DRUG_SEQ number (secondary key) are needed.  

From ENDNOTES in the documentation:  
> DRUG_SEQ (drug sequence number found in the Drug file, Therapy file, and Indications file) denotes the relationship between the drug(s) reported for a Case, the therapy date(s) reported for the drug(s), and the indications reported for the drug(s).  
Consider Case 3078140 version 1, received by the FDA on 12/31/97. The
PRIMARYID for this case is 30781401. Like any Case, it appears once (and only once) in the Demographic file:  
>
> | PRIMARYID |
  | ----- |
  | 30781401 |
>
>        
> Four drugs were reported for this Case: Aricept was reported as suspect, and Estrogens, Prozac, and Synthroid as concomitant. Primaryid 30781401 appears four times in the Drug file, with a different DRUG_SEQ for each drug:
>
> | PRIMARYID | DRUG_SEQ | DRUGNAME
  | --------- | -------- | --------
  | 30781401  | 1        | Aricept
  | 30781401  | 2        | Estrogens
  | 30781401  | 3        | Prozac( Fluoxetine Hydrochloride
  | 30781401  | 4        | Synthroid (Levothyroxine Sodium)
>
> Dates of therapy for Aricept were reported as "4/97 to 6/13/97", and "6/20/97 (ongoing)." Since the drug was started, stopped, then restarted, there are two entries in the Drug Therapy file. In such a circumstance, the two entries will have the same PRIMARYID and the same DRUG_SEQ # (or DSG_DRUG_SEQ number as it is called in the Therapy file - see below). No therapy dates were reported for the concomitants; therefore, they do not appear in the Drug Therapy file, which is excerpted as follows:  
>
> | PRIMARYID | DSG_DRUG_SEQ # | START_DT | END_DT
  | --------- | -------------- | -------- | ------
  | 30781401  | 1              | 199704   | 19970613
  | 30781401  | 1              | 19970620 |
>
> NOTE:  The Drug Seq number is no longer a unique key as was the case in LAERS QDE.  The Drug Seq number simply shows the order of the DRUGNAME within a unique case.  Additionally, the fields labeled DRUG_SEQ, INDI_DRUG_SEQ, and DSG_DRUG_SEQ in the Drug, Indication, and Therapy files, respectively, all serve the same purpose of linking the data elements in each individual file together with the appropriate drug listed in the case using the PRIMARYID.

In [285]:
drug.drug_seq.describe()

count   1546835.00
mean          7.20
std          11.60
min           1.00
25%           1.00
50%           4.00
75%           8.00
max         310.00
Name: drug_seq, dtype: float64

In [339]:
show_value_counts(drug.drug_seq)

,drug_seq,count,proportion
0,1,394061,0.25
1,2,211759,0.14
2,3,148953,0.10
3,4,115906,0.07
4,5,94757,0.06
5,6,78364,0.05
6,7,66161,0.04
7,8,55679,0.04
8,9,47721,0.03
9,10,40166,0.03


No missing values. Max drugs per case is 310. 

##### role_cod  
From documentation:  
> Code for drug's reported role in event (See table below)
>
> | CODE      | MEANING_TEXT
 | ----      | ------------
 | PS        | Primary Suspect Drug
 | SS        | Secondary Suspect Drug
 | C         | Concomitant
 | I         | Interacting

In [340]:
drug.role_cod.describe()

count     1546835
unique          4
top             C
freq       714731
Name: role_cod, dtype: object

In [341]:
show_value_counts(drug.role_cod)

,role_cod,count,proportion
0,C,714731,0.46
1,SS,428672,0.28
2,PS,394065,0.25
3,I,9367,0.01


In [351]:
show_na(drug.role_cod)

,na_count,na_proportion
0,0,0.00


No missing values. The frequency counts and proportions are consistent with the FDA numbers.

##### drugname  
From documentation:  
> Name of medicinal product.  
> If a "Valid Trade Name" is populated for this Case, then DRUGNAME = Valid Trade Name; if not, then DRUGNAME = "Verbatim" name, exactly as entered on the report.

In [352]:
drug.drugname.describe()

count      1546823
unique       61846
top       REVLIMID
freq         15475
Name: drugname, dtype: object

In [354]:
show_value_counts(drug.drugname, 20)

,drugname,count,proportion
0,REVLIMID,15475,0.01
1,HUMIRA,15081,0.01
2,ENBREL,13311,0.01
3,PREDNISONE.,12837,0.01
4,METHOTREXATE.,12210,0.01
5,XARELTO,11786,0.01
6,LYRICA,10128,0.01
7,REPATHA,9924,0.01
8,COSENTYX,9603,0.01
9,XOLAIR,9540,0.01


In [355]:
show_na(drug.drugname)

,na_count,na_proportion
0,12,0.00


In [357]:
drug[drug.drugname.isna()]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
665436,154818901,15481890,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
669087,154831781,15483178,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
722274,154998041,15499804,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
722275,154998041,15499804,3,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
722276,154998041,15499804,4,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
942462,155793001,15579300,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
1008975,156016371,15601637,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
1057196,156172351,15617235,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
1061749,156186731,15618673,3,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN
1105598,156319821,15631982,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN


In [358]:
drug[drug.primaryid == "154818901"]

,primaryid,caseid,drug_seq,role_cod,drugname,prod_ai,val_vbm,route,dose_vbm,cum_dose_chr,cum_dose_unit,dechal,rechal,lot_num,exp_dt,nda_num,dose_amt,dose_unit,dose_form,dose_freq
665435,154818901,15481890,1,PS,HUMIRA,ADALIMUMAB,1,Subcutaneous,? OTHER FREQUENCY:Q 2 WEEKS;?,nan,NaN,D,D,1095254,20191130,NaN,40.00,MG,NaN,QOW
665436,154818901,15481890,2,C,NaN,UNSPECIFIED INGREDIENT,1,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN


In [359]:
demo[demo.primaryid == "154818901"]

,primaryid,caseid,caseversion,i_f_code,event_dt,mfr_dt,init_fda_dt,fda_dt,rept_cod,auth_num,mfr_num,mfr_sndr,lit_ref,age,age_cod,age_grp,sex,e_sub,wt,wt_cod,rept_dt,to_mfr,occp_cod,reporter_country,occr_country
114413,154818901,15481890,1,I,NaN,NaN,20181003,20181003,DIR,NaN,NaN,FDA-CTU,NaN,nan,NaN,NaN,F,N,nan,NaN,20181003,N,CN,US,US


In [360]:
ascii_file_drug

'data/raw/ascii_2018q4/ascii/DRUG18Q4.txt'

In [362]:
# check the datalines in the raw input file
# look at line 665436 and surrounding lines
# looking for primaryid=154818901
!sed '665434,665440!d' data/raw/ascii_2018q4/ascii/DRUG18Q4.txt

154818882$15481888$9$SS$NOXAFIL$POSACONAZOLE$1$Oral$300 MG, QD$2400$MG$Y$$$$$300$MG$GASTRO-RESISTANT TABLET$QD
154818882$15481888$10$SS$ZAVEDOS$IDARUBICIN$1$Intravenous (not otherwise specified)$13.7 MG, QD$55$MG$$$$$$13.7$MG$$QD
154818891$15481889$1$PS$LAMICTAL$LAMOTRIGINE$1$Oral$200 MG, QD$$$Y$$$$020241$200$MG$TABLET$QD
154818901$15481890$1$PS$HUMIRA$ADALIMUMAB$1$Subcutaneous$?          OTHER FREQUENCY:Q 2 WEEKS;?$$$D$D$ 1095254$20191130$$40$MG$$QOW
154818901$15481890$2$C$N/A$UNSPECIFIED INGREDIENT$1$$$$$$$$$$$$$
154818922$15481892$1$PS$TYSABRI$NATALIZUMAB$1$Intravenous (not otherwise specified)$INFUSED OVER 1 HOUR$$$$U$$$$300$MG$CONCENTRATE FOR SOLUTION FOR INFUSION$
154818931$15481893$1$PS$SYMDEKO$IVACAFTOR\TEZACAFTOR$1$Oral$TEZACAFTOR/IVACAFTOR AM, IVACAFTOR PM$$$$$ 1540373$$210491$$$TABLET$


There are 12 missing values present in the raw data, and all of them have `prod_ai`="UNSPECIFIED INGREDIENT". This missing values count is inconsistent with the FDA's number of 0 in the accompanying documentation.  

##### prod_ai  
From documentation:  
> Product Active Ingredient, when available.  
>
> \* New tag added in 2014Q3 extract.

In [363]:
drug.prod_ai.describe()

count     1511985
unique       5782
top       ASPIRIN
freq        19377
Name: prod_ai, dtype: object

In [365]:
show_value_counts(drug.prod_ai, 10)

,prod_ai,count,proportion
0,NaN,34850,0.02
1,ASPIRIN,19377,0.01
2,LENALIDOMIDE,16440,0.01
3,ADALIMUMAB,15714,0.01
4,ACETAMINOPHEN,15582,0.01
5,DOCETAXEL,14951,0.01
6,PREDNISONE,14313,0.01
7,ETANERCEPT,14101,0.01
8,METHOTREXATE,13535,0.01
9,METFORMIN HYDROCHLORIDE,12457,0.01


About 2% of the values are missing. The missing values count is consistent with the FDA number.

In [284]:
drug.columns

Index(['primaryid', 'caseid', 'drug_seq', 'role_cod', 'drugname', 'prod_ai',
       'val_vbm', 'route', 'dose_vbm', 'cum_dose_chr', 'cum_dose_unit',
       'dechal', 'rechal', 'lot_num', 'exp_dt', 'nda_num', 'dose_amt',
       'dose_unit', 'dose_form', 'dose_freq'],
      dtype='object')

### 5. DQA summary

### 6. Next steps